### Tests

We're about to test our own MCP Server.

In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown
import json

load_dotenv(override=True)

True

[non-fatal] Tracing: server error 500, retrying.


### Now we write an MCP server and use it directly!

In [3]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "mcp_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()


In [4]:
mcp_tools

[Tool(name='valid_beckn_payload', description='Tell if the given payload is valid as per the Beckn Protocol Specification.\n\n    Args:\n        payload: The json payload to be validated\n    ', inputSchema={'properties': {'payload': {'additionalProperties': True, 'title': 'Payload', 'type': 'object'}}, 'required': ['payload'], 'title': 'valid_beckn_payloadArguments', 'type': 'object'}, annotations=None)]

In [47]:
instructions = "User are a helpful assistant who helps validate provide \
    if a provided JSON is valid as per Beckn protocol Specification. \
    Use the provided tools only to check the validity. \
    dont use any other source of truth to check validity. \
    "
request = "check if the below JSON is valid as per Beckn protocol Specification. "
model = "litellm/gemini/gemini-1.5-pro"
#model = "gpt-4o-mini"


# text = "if there is an internal server error in the tool, politely communicate to \
#         the user that the validation engine has an issue. Ask them to try again\
#     other wise return the reason if a JSON is invalid.\
#         dont return the stack trace of the error."



In [33]:
beckn_payload = {
    "context": {
        "domain": "local-retail",
        "location": {
            "country": {
                "code": "IND"
            },
            "city": {
                "code": "std:080"
            }
        },
        "action": "search",
        "version": "1.1.0",
        "bap_id": "farmfresh.bap-id.com",
        "bap_uri": "https://www.55a6-124-123-32-28.ngrok-free.app",
        "transaction_id": "8100d125-76a7-4588-88be-81b97657cd09",
        "message_id": "6104c0a3-d1d1-4ded-aaa4-76e4caf727ce",
        "timestamp": "2023-11-06T09:41:09.673Z"
    },
    "message": {
        "intent": {
            "item": {
                "descriptor": {
                    "name": "Name of product/service"
                }
            },
            "category": {
                "descriptor": {
                    "code": "electronics"
                }
            },
            "fulfillment": {
                "type": "DELIVERY",
                "stops": [
                    {
                        "location": {
                            "gps": "28.4594965,77.0266383"
                        }
                    }
                ]
            }
        }
    }
}

request += json.dumps(beckn_payload)

print(request)

check if the below JSON is valid as per Beckn protocol Specification. {"context": {"domain": "local-retail", "location": {"country": {"code": "IND"}, "city": {"code": "std:080"}}, "action": "search", "version": "1.1.0", "bap_id": "farmfresh.bap-id.com", "bap_uri": "https://www.55a6-124-123-32-28.ngrok-free.app", "transaction_id": "8100d125-76a7-4588-88be-81b97657cd09", "message_id": "6104c0a3-d1d1-4ded-aaa4-76e4caf727ce", "timestamp": "2023-11-06T09:41:09.673Z"}, "message": {"intent": {"item": {"descriptor": {"name": "Name of product/service"}}, "category": {"descriptor": {"code": "electronics"}}, "fulfillment": {"type": "DELIVERY", "stops": [{"location": {"gps": "28.4594965,77.0266383"}}]}}}}


In [40]:


async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="specification_expert", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("specification_expert"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


The provided payload is valid as per the Beckn Protocol Specification version 1.1.0.


In [63]:
request = "check if the below JSON is valid as per Beckn protocol Specification. "

In [64]:
beckn_payload = {
    "context": {
        "domain": "local-retail",
        "location": {
            "country": {
                "code": "IND"
            },
            "city": {
                "code": "std:080"
            }
        },
        "action": "search",
        "version": "1.1.0",
        "bap_id": "farmfresh.bap-id.com",
        "bap_uri": "https://www.55a6-124-123-32-28.ngrok-free.app",
        "transaction_id": "8100d125-76a7-4588-88be-81b97657cd09",
        "message_id": "6104c0a3-d1d1-4ded-aaa4-76e4caf727ce",
        "timestamp": "2023-11-06T09:41:09.673Z"
    },
    "message": {
        "intent": {
            "item": {
                "descriptor": {
                    "name": "Name of product/service"
                }
            },
            "tags": [
                {
                    "descriptor": {
                        "code": "BUYER_FINDER_FEES"
                    },
                    "list": [
                        {
                            "descriptor": {
                                "code": "BUYER_FINDER_FEES_PERCENTAGE"
                            },
                            "value": "200.00"
                        }
                    ]
                }
            ],
            "category": {
                "descriptor": {
                    "code": "electronics"
                }
            },
            "fulfillment": {
                "type": "DELIVERY",
                "stops": [
                    {
                        "location": {
                            "gps": "28.4594965,77.0266383"
                        }
                    }
                ]
            }
        }
    }
}

request += json.dumps(beckn_payload)

print(request)

check if the below JSON is valid as per Beckn protocol Specification. {"context": {"domain": "local-retail", "location": {"country": {"code": "IND"}, "city": {"code": "std:080"}}, "action": "search", "version": "1.1.0", "bap_id": "farmfresh.bap-id.com", "bap_uri": "https://www.55a6-124-123-32-28.ngrok-free.app", "transaction_id": "8100d125-76a7-4588-88be-81b97657cd09", "message_id": "6104c0a3-d1d1-4ded-aaa4-76e4caf727ce", "timestamp": "2023-11-06T09:41:09.673Z"}, "message": {"intent": {"item": {"descriptor": {"name": "Name of product/service"}}, "tags": [{"descriptor": {"code": "BUYER_FINDER_FEES"}, "list": [{"descriptor": {"code": "BUYER_FINDER_FEES_PERCENTAGE"}, "value": "200.00"}]}], "category": {"descriptor": {"code": "electronics"}}, "fulfillment": {"type": "DELIVERY", "stops": [{"location": {"gps": "28.4594965,77.0266383"}}]}}}}


In [65]:


async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="specification_expert", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("specification_expert"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


The provided JSON is not a valid Beckn payload. The error message indicates that the value associated with "BUYER_FINDER_FEES_PERCENTAGE" does not adhere to the specified pattern. Specifically, the value "200.00" violates the regex pattern which likely restricts it to a maximum of "99.99".


### Now let's build our own MCP Client

In [ ]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

In [ ]:
agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
result = await Runner.run(agent, request)
display(Markdown(result.final_output))

In [ ]:
context = await read_accounts_resource("ed")
print(context)

In [ ]:
from accounts import Account
Account.get("ed").report()